In [1]:
# -*- coding: utf-8 -*-
import json
from flask import Flask, render_template
from pyecharts import options as opts
from pyecharts.charts import Map
from flask.json import jsonify
from flask import Flask, render_template, request
from flask import Flask
from jinja2 import Markup, Environment, FileSystemLoader
from pyecharts.globals import CurrentConfig
import random
import time
from flask_cors import CORS
from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType
import requests
import json



In [2]:
def solve_scal(x):
    sort_list = ['1-49人',
                 '50-99人',
                 '100-499人',
                 '500-999人',
                 '1000-2000人',
                 '2000-5000人',
                 '10000人以上',
                 '5000-10000人'
                 ]

    empty_list = []
    new_list = []
    for y in x:
        if y == sort_list[0]:
            empty_list.append(int(0))
        elif y == sort_list[1]:
            empty_list.append(int(1))
        elif y == sort_list[2]:
            empty_list.append(int(2))
        elif y == sort_list[3]:
            empty_list.append(int(3))
        elif y == sort_list[4]:
            empty_list.append(int(4))
        elif y == sort_list[5]:
            empty_list.append(int(5))
        elif y == sort_list[6]:
            empty_list.append(int(6))
        elif y == sort_list[7]:
            empty_list.append(int(7))
    get_list = sorted(empty_list)
    for i in range(len(get_list)):
        new_list.append(sort_list[get_list[i]])
    return new_list

In [12]:
def create_chart(job_name, city_name):
    import pandas
    from requests_html import HTMLSession
    import json
    import pandas as pd
    import requests
    用户输入职位 = job_name
    用户输入城市 = city_name
    城市编码 = {
        '全国': '410',
        '北京': '010',
        '上海': '020',
        '天津': '030',
        '重庆': '040',
        '广州': '050020',
        '深圳': '050090',
        '苏州': '060080',
        '南京': '060020',
        '杭州': '070020',
        '大连': '210040',
        '成都': '280020',
        '武汉': '170020',
        '西安': '270020'

    }
    # workExperiences = [{'code': '1', 'name': '应届生'},
    #                    {'code': '2', 'name': '实习生'},
    #                    {'code': '0$1', 'name': '1年以内'},
    #                    {'code': '1$3', 'name': '1-3年'},
    #                    {'code': '3$5', 'name': '3-5年'},
    #                    {'code': '5$10', 'name': '5-10年'},
    #                    {'code': '10$999', 'name': '10年以上'}]


    url = "https://apic.liepin.com/api/com.liepin.searchfront4c.pc-search-job"
    payload = {
        "data": {
            "mainSearchPcConditionForm": {
                "city": 城市编码[用户输入城市],
                "dq": 城市编码[用户输入城市],
                "pubTime": "",
                "currentPage": 0,
                "pageSize": 40,
                "key": str(用户输入职位),
                "suggestTag": "",
                "workYearCode": '',
                "compId": "",
                "compName": "",
                "compTag": "",
                "industry": "",
                "salary": "",
                "jobKind": "",
                "compScale": "",
                "compKind": "",
                "compStage": "",
                "eduLevel": ""
            },
            "passThroughForm": {
                "scene": "input",
                "skId": "",
                "fkId": "",
                "ckId": "",
                "suggest": None
            }
        }
    }
    session = HTMLSession()
    headers = {}
    r = session.post(url, data=json.dumps(payload), headers=headers)
    response_data = r.json()
    page = response_data['data']['pagination']['totalPage']
    response_df = []
    import time
    import random
    for i in range(3): # 需要判断页面的数据有多少页
        payload['data']['mainSearchPcConditionForm']['currentPage']=i
        r = session.post(url, data=json.dumps(payload), headers=headers)
        response_data = r.json()
        df = pd.json_normalize(response_data['data']['data']['jobCardList'])
        response_df.append(df)
        wait_time = random.randint(3,10)
        time.sleep(wait_time)
        print('打印到第'+str(i)+'页')
    response_df = []
    df = pd.json_normalize(response_data['data']['data']['jobCardList'])
    response_df.append(df)
    df = pd.concat(response_df)
    key = payload['data']['mainSearchPcConditionForm']['key']
    df.to_excel('liepin2.xlsx')
    df = pd.read_excel('liepin2.xlsx')
    df_all = pd.read_excel('liepin2.xlsx')
    df_all
    df_cities = df[['job.dq', 'comp.compScale', 'comp.compIndustry', 'comp.compName', 'job.title', 'job.salary']]
    df_cities
    df_cities = df_cities.rename(columns={
        'job.dq': '地区',
        'comp.compScale': '公司规模',
        'comp.compIndustry': '公司行业',
        'comp.compName': '公司名称',
        'job.title': '岗位名称',
        'job.salary': '工资',

    })
    set_dq = set(df_cities['地区'])
    list_dq = list(set_dq)
    list_dq
    data_all = [{
        'name': city_name,
        'children': [
            {'name': a,
             'children': [
                 {
                     'name': "公司规模:" + b,
                     'children': [
                         {
                             'name': "公司行业:" + str(c),
                             'children': [
                                 {
                                     'name': "公司名称:" + d,
                                     'children': [
                                         {
                                             'name': "岗位名称:" + e,
                                             'children': [
                                                 {
                                                     'name': "工资：" + f,
                                                 } for f in list(set(df_cities[(df_cities['地区'] == a) & (
                                                             df_cities['公司规模'] == b) & (df_cities[
                                                                                                '公司行业'] == c) & (
                                                                                           df_cities[
                                                                                               '公司名称'] == d) & (
                                                                                           df_cities['岗位名称'] == e)][
                                                                         '工资']))
                                             ]
                                         } for e in list(set(df_cities[(df_cities['地区'] == a) & (
                                                     df_cities['公司规模'] == b) & (df_cities['公司行业'] == c) & (
                                                                                   df_cities['公司名称'] == d)][
                                                                 '岗位名称']))
                                     ]
                                 } for d in list(set(df_cities[
                                                         (df_cities['地区'] == a) & (df_cities['公司规模'] == b) & (
                                                                     df_cities['公司行业'] == c)]['公司名称']))
                             ]
                         } for c in
                         list(set(df_cities[(df_cities['地区'] == a) & (df_cities['公司规模'] == b)]['公司行业']))
                     ]
                 } for b in solve_scal(list(set(df_cities[(df_cities['地区'] == a)]['公司规模'])))
             ]
             } for a in list(set(df_cities['地区']))
        ]
    }
    ]
    data_all
    if payload['data']['mainSearchPcConditionForm']["workYearCode"] in ['1', '2']:
        df_PM_gz = df[
            ['job.labels', 'job.refreshTime', 'job.title', 'job.salary', 'job.dq', 'job.topJob', 'comp.compStage',
             'comp.compName', 'comp.compIndustry', 'comp.compScale']]
    else:
        df_PM_gz = df[
            ['job.labels', 'job.refreshTime', 'job.title', 'job.salary', 'job.dq', 'job.topJob', 'job.requireWorkYears',
             'job.requireEduLevel', 'comp.compStage', 'comp.compName', 'comp.compIndustry', 'comp.compScale']]

    df_PM_gz['job.dq'].value_counts()
    地区 = [df_PM_gz['job.dq'].value_counts().index.tolist()[i].split('-')[1] \
            for i, v in enumerate(df_PM_gz['job.dq'].value_counts().index.tolist()) if '-' in v]
    地区
    岗位个数 = [df_PM_gz['job.dq'].value_counts().values.tolist()[i] for i, v in
                enumerate(df_PM_gz['job.dq'].value_counts().index.tolist()) if "-" in v]
    岗位个数
    df_PM_gz['job.title'][df_PM_gz['job.title'].str.contains('（')].str.split('（').apply(lambda x: x[0])
    df_job_title = df_PM_gz['job.title'].apply(lambda x: x.split('（')[0].split('/')[0].split('(')[0]).value_counts()
    df_job_title.index.tolist()
    len(df_job_title.index.tolist())
    df_job_title.values.tolist()
    df_PM_gz['job.title'].value_counts()
    PM_title_words = [(df_job_title.index.tolist()[i], df_job_title.values.tolist()[i]) for i in
                      range(1, len(df_job_title.index.tolist()))]
    PM_title_words
    df_PM_gz['job.labels']
    df_PM_gz['job.labels'].values
    df_PM_gz['job.labels'].apply(lambda x: eval(x)).tolist()
    PM_labels_list = [j for i in df_PM_gz['job.labels'].apply(lambda x: eval(x)).tolist() for j in i]
    PM_labels_list
    PM_labels_words = [(i, PM_labels_list.count(i)) for i in set(PM_labels_list)]
    PM_labels_words
    if payload['data']['mainSearchPcConditionForm']["workYearCode"] in ['1', '2']:
        df_PM_gz = df_PM_gz.rename(columns={
            'job.labels': '职位标签',
            'job.refreshTime': '职位更新时间',
            'job.title': '职位',
            'job.salary': '薪资',
            'job.dq': '地区',
            'job.topJob': '是否top职位',
            'comp.compStage': '公司融资情况',
            'comp.compName': '公司名称',
            'comp.compIndustry': '行业',
            'comp.compScale': '规模'
        })
    else:
        df_PM_gz = df_PM_gz.rename(columns={
            'job.labels': '职位标签',
            'job.refreshTime': '职位更新时间',
            'job.title': '职位',
            'job.salary': '薪资',
            'job.dq': '地区',
            'job.topJob': '是否top职位',
            'job.requireWorkYears': '工作年限',
            'job.requireEduLevel': '学历',
            'comp.compStage': '公司融资情况',
            'comp.compName': '公司名称',
            'comp.compIndustry': '行业',
            'comp.compScale': '规模'
        })
    if payload['data']['mainSearchPcConditionForm']["workYearCode"] in ['1', '2']:
        非薪资面议 = df_PM_gz[~df_PM_gz['薪资'].str.contains("面议")]

    else:
        非薪资面议 = df_PM_gz[~df_PM_gz['薪资'].str.contains("面议|元/天")]

    if payload['data']['mainSearchPcConditionForm']["workYearCode"] in ['2']:
        非薪资面议_detail = 非薪资面议['薪资'].apply(lambda x: x.split('元')[0]).tolist()
        平均薪资 = [round((int(i.split('-')[0]) + int(i.split('-')[1])) / 2 * 30) \
                        if i.__contains__('-') else int(i) * 30 \
                    for i in 非薪资面议_detail]
    else:
        非薪资面议_detail = 非薪资面议['薪资'].apply(lambda x: x.split('薪')[0].split('·')[0]).tolist()
        平均薪资 = [(int(i.split('-')[0]) + int(i.split('-')[1].split('k')[0])) / 2 \
                        if len(i) == 1 else round(
            (int(i.split('-')[0]) + int(i.split('-')[1].split('k')[0])) / 2 * 1000) \
                    for i in 非薪资面议_detail]

    非薪资面议['平均薪资'] = 平均薪资
    分地区_平均薪资 = 非薪资面议.groupby('地区').agg({'平均薪资': 'median'})
    分地区_平均薪资_values = [round(i[0], 1) for i in 分地区_平均薪资.values.tolist()]
    分地区_平均薪资_values
    分地区_平均薪资_index = 分地区_平均薪资.index.tolist()
    分地区_平均薪资_index
    from pyecharts import options as opts
    from pyecharts.charts import Tree
    import time
    time.localtime()
    output_time = str(time.localtime().tm_year) + '-' \
              + str(time.localtime().tm_mon) + '-' \
              + str(time.localtime().tm_mday)
    output_time
    a = (
        Tree()
        .add("", data_all, collapse_interval=1, layout="radial")
        .set_global_opts(title_opts=opts.TitleOpts(title=city_name + '-' + job_name + '-' + "岗位速查树状图"))
        .render(f'charts/{city_name}_{job_name}_{output_time}_tree.html')
    )
    b = (
        Map()
        .add(str(city_name), [list(z) for z in zip(地区, 岗位个数)], str(city_name))
        .set_global_opts(
            title_opts=opts.TitleOpts(title=city_name + '-' + job_name + '-' "岗位分布地图"),
            visualmap_opts=opts.VisualMapOpts()
        )
        .render(f'charts/{city_name}_{job_name}_{output_time}_map.html')
    )
    d = (
        WordCloud()
        .add("", PM_title_words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
        .set_global_opts(title_opts=opts.TitleOpts(title=city_name + '-' + job_name + '-' + "热门岗位词云图"))
        .render(f'charts/{city_name}_{job_name}_{output_time}_word.html')
    )


In [13]:
job_list = ["新媒体编辑", "社交媒体经营", "数据分析师", "新媒体营销专员", "内容运营", "数字营销专家", "用户体验设计师", "社交媒体经理", "数字艺术家", "内容创作者"]
cities_list = ["北京", "上海", "天津", "重庆", "广州", "深圳", "苏州", "南京", "杭州", "大连", "成都", "武汉", "西安"]
import random
wait_time = random.randint(50,60)
for x in job_list:
    for y in cities_list:
        create_chart(x,y)
        print(str(x)+'-'+str(y))
        time.sleep(wait_time)

打印到第0页
打印到第1页
打印到第2页
北京
打印到第0页
打印到第1页
打印到第2页
上海


C:\Users\17728\AppData\Local\Temp\ipykernel_26120\2882721627.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  非薪资面议['平均薪资'] = 平均薪资


打印到第0页
打印到第1页
打印到第2页
天津


C:\Users\17728\AppData\Local\Temp\ipykernel_26120\2882721627.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  非薪资面议['平均薪资'] = 平均薪资


打印到第0页
打印到第1页
打印到第2页
重庆


...

C:\Users\17728\AppData\Local\Temp\ipykernel_26120\2882721627.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  非薪资面议['平均薪资'] = 平均薪资


苏州
打印到第0页
打印到第1页
打印到第2页


C:\Users\17728\AppData\Local\Temp\ipykernel_26120\2882721627.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  非薪资面议['平均薪资'] = 平均薪资


南京


KeyboardInterrupt: 